# **Project overview**
The Enigma machine is a famous encryption device used by the Germans, most notably during World War II, for securing military communications. The machine resembles a typewriter and operates by inputting letters through a keyboard, which are then encrypted based on the configuration of several rotors and a plugboard before being output as a different letter.

This setup allowed for a vast number of possible settings, making the Enigma's encryption extremely difficult to break.  
We will try to Break the enigma code using modern tools and not just in terms of computing power.  
We will use spesificlly machine learning tools: AdaBoost, Random Forest, SVM and Neural Network (more precisely LSTM)

In [ ]:
import numpy as np
import time


# Adjusted char_to_int function to handle a broader range of characters safely
def char_to_int(char):
    if 'a' <= char <= 'z':
        return ord(char) - ord('a')
    # Add more conditions here for other characters in your dataset
    elif char == ' ':
      print("char found!")
      return 26
    # Handle unexpected characters
    else:
      print("unexpected characters found!")
      return 27

# Function to define int to char (inverse of char_to_int)
def int_to_char(i):
    return chr(i + ord('a'))

# **Data Preparation**
The Data is a collection of words and their encoding.
every word starts in the same state of the machine so the model will have to understand the relations between the Enigma parts in order to understand the current configuration.



In [ ]:

# Path to your dataset
# file_path = '/content/sample_data/data_bi_direction.txt'
file_path = '/content/sample_data/data.txt'

# Read the dataset
with open(file_path, 'r') as file:
    lines = file.readlines()
    data = [line.strip().split(' ') for line in lines]


# Extract origin words and encoded words
origin_words = [item[0] for item in data]
encoded_words = [item[1] for item in data]

# Encode words as sequences of integers
encoded_origin_words = [[char_to_int(char) for char in word] for word in origin_words]
encoded_encoded_words = [[char_to_int(char) for char in word] for word in encoded_words]

max_word_length = max(max(len(word) for word in origin_words), max(len(word) for word in encoded_words))


# **AdaBoost - Using Decision Tree**
First we will try to use AdaBoost - It's seems requested since the algorithm combines multiple weak learners to create a strong learner, so we can use each letter permution in a given level (letter index) as a weak rule - combine them and mabey the algorithem will mannage to learn the Enigma configuration pattern

There is still one issue we can think about it - even in case this algorithm will work perfectly, It still always be blocked on the longest word in the Dataset - since that our rules are set only for the max size of the Dataset.

## **Step 1: Data Preprocessing**

AdaBoost typically deals with fixed-length feature vectors, so we'll need to address the variable length of words -> our approach is to pad sequences to have the same length, ensuring that all input vectors are of equal length.

Be aware that we are dealing with a classification problem where each position in a word could potentially be classified into one of 28 classes (26 letters + space + unexpected characters).

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Function to pad encoded word sequences to the same length
def pad_encoded_sequences(encoded_sequences, max_length):
    return [seq + [27] * (max_length - len(seq)) for seq in encoded_sequences]  # 27 is used for padding

# Pad the encoded sequences
padded_encoded_origin_words = pad_encoded_sequences(encoded_origin_words, max_word_length)
padded_encoded_encoded_words = pad_encoded_sequences(encoded_encoded_words, max_word_length)

## **Step 2: Splitting the Data**
Split our data into a training set and a test set.

In [ ]:
# Assuming padded_encoded_origin_words is your input data and padded_encoded_encoded_words is your target data
X_train, X_test, y_train, y_test = train_test_split(padded_encoded_origin_words, padded_encoded_encoded_words, test_size=0.2, random_state=42)


## **Step 3: Model Building and Training**
Train an AdaBoost classifier for each position using the training data - For each position in the word, train a separate AdaBoost classifier.


In [ ]:
classifiers = []  # store the classifiers for each position

# timer - start
adaboost_time_start = time.time()

# max_word_length is the maximum word length
for position in range(max_word_length):
    # Prepare the dataset for the current position
    X = np.array([word[position] for word in X_train if position < len(word)])
    y = np.array([word[position] for word in y_train if position < len(word)])

    # Initialize and train the AdaBoost classifier for the current position
    clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=50)
    clf.fit(X.reshape(-1, 1), y)  # X needs to be reshaped because scikit-learn expects 2D arrays for X

    classifiers.append(clf)
    # Now, classifiers[i] is the AdaBoost model for the i-th position in the word.

# timer - end
adaboost_time_end = time.time()

## **Step 4: Evaluation**
For each position, predict the encoded character using the test set and calculate the accuracy.


In [ ]:
from sklearn.metrics import accuracy_score

accuracies = []  # To store the accuracy of each classifier

for position in range(max_word_length):
    # Extract the test data for the current position
    X_test_position = np.array([word[position] for word in X_test if position < len(word)])
    y_test_position = np.array([word[position] for word in y_test if position < len(word)])

    # Predict using the classifier for the current position
    y_pred = classifiers[position].predict(X_test_position.reshape(-1, 1))

    # Calculate the accuracy for the current position
    accuracy = accuracy_score(y_test_position, y_pred)
    accuracies.append(accuracy)

# Calculate the overall accuracy
overall_accuracy = np.mean(accuracies)
print(f"AdaBoost Accuracy: {overall_accuracy}")
print(f'AdaBoost fit time: {adaboost_time_end - adaboost_time_start}')

AdaBoost Accuracy: 0.9295777646903634
AdaBoost fit time: 393.9909701347351


# **Regression - Random Forest Regressor**
Second we will try Regression. Since this problem is not involve predicting a continuous target variable - but predicting many variables together (sequence of letters encoding) we will need ***Multi-output Regression***, where the goal is to predict multiple target variables instead of just one.

This method is a good pick since Multi-output Regression works best when variables are interrelated - and in our case the enigma changes it's configuration in each index (due to the rotors non stop movment) so in each letter you would exepect a different output.

After some tests with Multi-output Regression model we found **Random Forest Regressor** is the best suit for our task - it's an ensemble learning technique that combines multiple decision trees to provide an accurate predictions for regression tasks.

## **Step 1: Data Preprocessing**
Pad the Encoded Words to ensure they all have the same length.

This step is necessary for creating a consistent input and output shape for our model.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import OneHotEncoder

# Function to pad the encoded words to the same length
def pad_encoded_words(encoded_words, max_length):
    return np.array([np.pad(word, (0, max_length - len(word)), 'constant', constant_values=27) for word in encoded_words])

# Apply padding
padded_encoded_origin_words = pad_encoded_words(encoded_origin_words, max_word_length)
padded_encoded_encoded_words = pad_encoded_words(encoded_encoded_words, max_word_length)

## **Step 2: One-Hot Encoding**
This step converts the integer-encoded characters into a binary matrix representation suitable for regression. Each character (including the padding) is represented as a binary vector with all zeros except for the index of the character, which is set to 1.

One-hot encoding is a process used to convert data into a format that can be provided to machine learning algorithms to do a better job in prediction. It works by converting each value into a new categorical column and assigns a 1 or 0 (True/False) value to those columns in each row.

In [ ]:
# One-hot encoding the data
encoder = OneHotEncoder()
encoded_inputs = encoder.fit_transform(padded_encoded_origin_words).toarray()
encoded_outputs = encoder.fit_transform(padded_encoded_encoded_words).toarray()

## **Step 3: Splitting the Data**
Split the Dataset into training and testing sets to evaluate the performance of the model.


In [ ]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_encoded_origin_words, padded_encoded_encoded_words, test_size=0.2, random_state=42)

## **Step 4: Model Definition and Training**
We use sklearn.ensemble.RandomForestRegressor - A random forest is a meta estimator that fits a number of decision tree regressors on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.



*   n_estimators - The number of trees in the forest.
*   random_state - Controls both the randomness of the bootstrapping of the samples used when building trees.



In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

regression_time_start = time.time()

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

regression_time_end = time.time()

## **Step 5: Evaluation**
 calculate accuracy

In [ ]:
# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
# Here, we use mean squared error as a simple evaluation metric, but you can choose metrics that best suit your needs.
mse = mean_squared_error(y_test, y_pred)
print(f'Regression Mean Squared Error: {mse}')

def rounded_accuracy(y_true, y_pred):
    # Round predictions to nearest integer
    rounded_preds = np.round(y_pred)
    # Calculate accuracy
    correct_predictions = np.sum(rounded_preds == y_true, axis=1)
    total_predictions = y_true.shape[1]
    # Average accuracy per word
    accuracy_per_word = correct_predictions / total_predictions
    # Overall accuracy
    overall_accuracy = np.mean(accuracy_per_word)
    return overall_accuracy

# Evaluate accuracy
accuracy = rounded_accuracy(y_test, y_pred)
print(f"Regression Accuracy: {accuracy}")
print(f'Regression Fit time: {regression_time_end - regression_time_start}')

Regression Mean Squared Error: 0.38119596401650485
Regression Accuracy: 0.9859030889496835
Regression Fit time: 145.946448802948


# **SVM**
Now, we experimenting indirectly use SVM for such a task (break the enigma), one unconventional approach could involve treating it as a *classification problem*, where each original letter maps to an encoded letter. This requires a significantly different setup.

First we tried an word to word encoding  - that was a problematic classifiction problem since In a typical classification or regression task, y is expected to be a 1D array, where each element corresponds to a single label or value per input sample. However, in our case, each input sample (a word) is associated with multiple labels (the encoded characters), making it a multilabel classification problem.

So We need a different approch...

We will try to Simplify the Problem - focusing on character-to-character encoding rather than whole words. This approach significantly reduces complexity.
This approach treats each character in the dataset as a separate data point, where the input feature is the original character, and the target output is the encoded character

## **Step 1: Data Preprocessing**


*   Flatten the dataset so that each character and its encoded counterpart form a data point.
*   Encode characters as integer values (Given the nature of SVM  we'll stick with integer encoding).



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

# since the data is pairs of chars - split the data for faster results
split_point = len(encoded_origin_words) // 8
print('len(encoded_origin_words): ', len(encoded_origin_words))
print('split_point: ', split_point)

# Use only the first half of each list
splited_encoded_origin_words = encoded_origin_words[:split_point]
splited_encoded_encoded_words = encoded_encoded_words[:split_point]

data_size = 0

for word in splited_encoded_origin_words:
  data_size += len(word)

print('SVM data set size: ', data_size)

# Flatten the dataset to character level
X = []
y = []
for original, encoded in zip(splited_encoded_origin_words, splited_encoded_encoded_words):
    X.extend(original)
    y.extend(encoded)

# Ensure X and y are numpy arrays
X = np.array(X).reshape(-1, 1)  # Reshape for sklearn compatibility
y = np.array(y)


len(encoded_origin_words):  418586
split_point:  52323
SVM data set size:  279274


## **Step 2: Splitting the Data**
Split the data into training and testing sets.

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## **Step 3: Model Building and Training**
Train an SVM model on the character-to-character mapping.

In [ ]:
svm_time_start = time.time()

# Train the SVM model
model = SVC(kernel='rbf', gamma='scale')  # 'scale' is usually a good default for gamma
model.fit(X_train, y_train)

svm_time_end = time.time()

## **Step 4: Evaluation**
Evaluate the model's performance on the test set to see how well it predicts the encoding for unseen characters.


In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'SVM Accuracy: {accuracy}')
print(f'SVM fit time: {svm_time_end - svm_time_start}')

SVM Accuracy: 0.298379733237848
SVM fit time: -1278.2595884799957


# **LSTM**
Lastly, After consulting with Liad that sugessted to look into RNN's models - we found the stateful LSTM model: To capture the evolving state of the Enigma machine, the stateful LSTM model seemed like the right choise.

Stateful LSTMs have the ability to maintain state across batches, meaning that the state of the machine after encrypting one word can be carried over to the next word, mimicking the continuous operation of the Enigma machine.

But This requireed careful batch preparation, ensuring that the sequence of words is maintained and that batches start where the previous batch left off.

## **Step 1: Data Preprocessing**

We need to encode the letters to numerical values and prepare the data for sequence modeling.

* Character Encoding: Map each unique character to an integer. Since the Enigma machine operates on letters, this will likely be a mapping of the alphabet.
* Sequence Creation: Create sequences of these integers. Given the Enigma machine's stateful nature, each input letter's encoding must be followed by its corresponding output letter's encoding.
* Data Structuring: For a stateful LSTM, it's crucial that the data fed into the model allows for the state to carry over from one batch to the next appropriately.


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.models import Sequential  # Import Sequential model
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed  # Ensure all layers are imported


# Pad sequences to the maximum word length to ensure uniform input size
X = pad_sequences(encoded_origin_words , maxlen=max_word_length, padding='post')
y = pad_sequences(encoded_encoded_words, maxlen=max_word_length, padding='post')

# Assuming the dataset might contain lowercase letters and potentially other characters.
chars = sorted(list(set(''.join(origin_words) + ''.join(encoded_words))))
num_classes = len(chars)  # This should include all unique characters

# Update y to categorical
y = np.array([to_categorical(encoded_word, num_classes=len(chars)) for encoded_word in y])


## **Step 2: Splitting the Data**

Split the dataset into training, validation, and test sets using scikit-learn's train_test_split function:

In [ ]:
# Split the data into training and test sets (80% train, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the remaining data into validation and test sets (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


## **Step 3: Model Building and Training**


Modify the model to fit it with the training data and validate using the validation data. Ensure to reset states appropriately.

We'll construct a simple stateful LSTM model using a deep learning framework Keras.

The model have:

1. An Embedding layer to convert integer representations of letters into dense vector embeddings.
2. A Stateful LSTM layer to capture the sequences' dependencies, considering the Enigma's rotor movement.
3. A Dense layer with a softmax activation to map the LSTM's output to a probability distribution over possible output letters.

In [ ]:
model = Sequential([
    Embedding(len(chars), 10, input_length=max_word_length),  # Adjust input_length to max_word_length
    LSTM(50, return_sequences=True),  # return_sequences=True for sequence prediction
    TimeDistributed(Dense(len(chars), activation='softmax'))  # Predict a sequence
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Adjust the training process to no longer require reshaping of X and resetting states after each batch

lstm_time_start = time.time()

# Example training call, adjust epochs and batch_size as needed
model.fit(X, y, epochs=2, batch_size=64, validation_split=0.2)

lstm_time_end = time.time()


Epoch 1/2
5233/5233 [==============================] - 116s 22ms/step - loss: 0.2106 - accuracy: 0.9463 - val_loss: 0.0306 - val_accuracy: 0.9925
Epoch 2/2
5233/5233 [==============================] - 114s 22ms/step - loss: 0.0338 - accuracy: 0.9910 - val_loss: 0.0110 - val_accuracy: 0.9974


## **Step 4: Evaluation**
Finally, evaluate the model's performance on the test dataset:



In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test, batch_size=1)
model.reset_states()  # Reset the states for the next evaluation or prediction
print(f'LSTM Test Loss: {loss}')
print(f'LSTM Accuracy: {accuracy}')
print(f'LSTM fit time: {lstm_time_end - lstm_time_start}')


41859/41859 [==============================] - 102s 2ms/step - loss: 0.0238 - accuracy: 0.9936
LSTM Test Loss: 0.02377822995185852
LSTM Accuracy: 0.993604302406311
LSTM fit time: 236.70498871803284


## **Step 5: Lets See The Model In Action**

In [ ]:
# Function to preprocess the input text
def preprocess_input(text):
  no_space_word = ""
  for i in range(len(text)):
    if(text[i] != ' '):
      no_space_word += text[i]
  encoded_word = [char_to_int(char) for char in no_space_word]
  padded_sequence = pad_sequences([encoded_word], maxlen=max_word_length, padding='post')
  return padded_sequence

# Prepare the input text
# machine = yolopws
input_text = "jcdhx"
preprocessed_input = preprocess_input(input_text)

# Make prediction using the model
predicted_sequence = model.predict(preprocessed_input)

# Decode the predicted sequence (optional)
def decode_sequence(sequence):
  decoded_word = [int_to_char(i.argmax()) for i in sequence.squeeze(0)]
  # print("decode_sequence: ", decoded_word)
  return ''.join(decoded_word)  # Remove brackets and trailing spaces

# Print the predicted sequence
predicted_word = decode_sequence(predicted_sequence)
print(f"Predicted word for '{input_text}': {predicted_word[:len(input_text)]}")

1/1 [==============================] - 0s 18ms/step
Predicted word for 'jcdhx': almog
